In [2]:
!python -m pip install --upgrade setuptools
!python -m pip install --upgrade pip
!python -m pip install nashpy
!python -m pip install quantecon

In [19]:
import nashpy as nash
import numpy as np

# Create the game with the payoff matrices (Matching Pennies)
# A: row player's payoffs; B: column player's payoffs (note: zero-sum => B = -A)

A = np.array([[ 1, -1],
              [-1,  1]])  # Row player (Heads/Tails)

B = np.array([[-1,  1],
              [ 1, -1]])  # Column player = -A

# Form the game
game1 = nash.Game(A, B)
game1


Zero sum game with payoff matrices:

Row player:
[[ 1 -1]
 [-1  1]]

Column player:
[[-1  1]
 [ 1 -1]]

**Interpretation.** The printed object confirms a two-player zero-sum normal-form game with row payoff matrix **A** and column payoff matrix **B = −A** — the standard **Matching Pennies** formulation. In this game there is **no pure-strategy Nash equilibrium**; instead the **unique equilibrium is mixed**, with each player randomizing **H** and **T** with probability **1/2** to keep the opponent indifferent. At that profile the row player’s expected payoff is **0** (and the column player’s is **−0**). This aligns with standard textbook treatments of Matching Pennies and mixed-strategy equilibria in finite games.





In [6]:
# Find the Nash Equilibrium with Support Enumeration
equilibria = game1.support_enumeration()
for eq in equilibria:
    print(eq)

(array([0.5, 0.5]), array([0.5, 0.5]))


**Interpretation:** The solver found the unique mixed-strategy Nash equilibrium of Matching Pennies: both players randomize (0.5, 0.5) over
𝐻
and
𝑇
, making each other indifferent; **expected payoffs are 0 in this zero-sum game.** This matches standard textbook results.

In [7]:
# import the QuantEcon package
import quantecon.game_theory as gt
import numpy as np

# --- The first way to form the game: by the payoff matrix of tuples ---
# (u_row, u_col)
matching_pennies_matrix = np.array([[( 1, -1), (-1,  1)],
                                    [(-1,  1), ( 1, -1)]])

g_MP = gt.NormalFormGame(matching_pennies_matrix)
print(g_MP)

2-player NormalFormGame with payoff profile array:
[[[ 1, -1],  [-1,  1]],
 [[-1,  1],  [ 1, -1]]]


**Interpretation:** The printout confirms a 2-player `NormalFormGame` where each cell lists a payoff pair (u_row, u_col). The displayed “payoff profile array” shows the bimatrix for Matching Pennies: the row player’s matrix [[1, −1], [−1, 1]] and the column player’s [[−1, 1], [1, −1]] = −A, i.e., a zero-sum specification; this indicates the game was constructed correctly for subsequent equilibrium computations. :contentReference[oaicite:0]{index=0}

In [8]:
# --- The second way to form the game: by players ---
# Player 0
player0 = gt.Player([[ 1, -1],
                     [-1,  1]])

# Player 1
player1 = gt.Player([[-1,  1],
                     [ 1, -1]])

# Pass in the two players
g_MP = gt.NormalFormGame((player0, player1))
print(g_MP)

2-player NormalFormGame with payoff profile array:
[[[ 1, -1],  [-1,  1]],
 [[-1,  1],  [ 1, -1]]]


**Interpretation:** This printout shows a 2-player `NormalFormGame` built from two `Player` objects. Each cell is a payoff pair (u_row, u_col); the row player’s matrix [[1, −1], [−1, 1]] and the column player’s [[−1, 1], [1, −1]] = −A confirm a zero-sum Matching Pennies bimatrix, so the game is correctly specified for equilibrium computation. (cf. QuantEcon `NormalFormGame`/`Player` docs on payoff arrays.)


In [14]:
# --- The third way to form the game: by actions (Not recommended) ---
# 2 players, each of whom has 2 actions
g_MP = gt.NormalFormGame((2, 2))

# (u_row, u_col)
g_MP[0, 0] =  1, -1   # (Row H, Col H)
g_MP[0, 1] = -1,  1   # (Row H, Col T)
g_MP[1, 0] = -1,  1   # (Row T, Col H)
g_MP[1, 1] =  1, -1   # (Row T, Col T)

print(g_MP)

2-player NormalFormGame with payoff profile array:
[[[ 1., -1.],  [-1.,  1.]],
 [[-1.,  1.],  [ 1., -1.]]]


**Interpretation**: This printout confirms a 2-player `NormalFormGame` created “by actions,” where each cell stores a payoff pair (u_row, u_col). The shown payoff profile array matches the Matching Pennies bimatrix: the row matrix [[1, −1], [−1, 1]] and the column matrix [[−1, 1], [1, −1]] = −A, i.e., a zero-sum specification. This indicates the game is correctly constructed and equivalent to the other two construction methods.

In [15]:
# --- Finding the Nash equilibrium: pure_nash_brute ---

NE = gt.pure_nash_brute(g_MP)
print("pure_nash_brute:", NE)

pure_nash_brute: []


**Interpretation**: The empty list means **no pure-strategy Nash equilibria** were found—exactly what we expect for **Matching Pennies**. Use a mixed-equilibrium routine next (e.g., support/vertex enumeration) to obtain the unique mixed NE.


In [17]:
# --- Finding the Nash equilibrium: support_enumeration ---
NE = gt.support_enumeration(g_MP)
print("support_enumeration:", NE)

support_enumeration: [(array([0.5, 0.5]), array([0.5, 0.5]))]


**Interpretation**: The solver (`support_enumeration`) returns the mixed-strategy Nash equilibrium for Matching Pennies: both players randomize (0.5, 0.5) over H and T, which is the unique NE in this zero-sum game; expected payoffs are 0.


In [16]:
# --- Finding the Nash equilibrium: vertex_enumeration ---
NE = gt.vertex_enumeration(g_MP)
print("vertex_enumeration:", NE)

vertex_enumeration: [(array([0.5, 0.5]), array([0.5, 0.5]))]


**Interpretation**: `vertex_enumeration` independently recovers the same mixed-strategy Nash equilibrium as support enumeration—both players mix (0.5, 0.5) over H and T. In QuantEcon, vertex enumeration searches best-response polytopes and (for non-degenerate 2-player games) returns all equilibria, so it matches the support-enumeration result here.
